In [1]:
from torch.utils.data import Dataset
import tensorflow_datasets as tfds
import tensorflow as tf
from typing import List
import numpy as np
from datasets import load_dataset
import regex as re
from tqdm import tqdm
from sys import stdout

2025-01-23 12:49:45.389599: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-23 12:49:45.405198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737665385.423887 2019935 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737665385.429580 2019935 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-23 12:49:45.448989: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
from lz78 import Sequence, CharacterMap, BlockLZ78Encoder, LZ78SPA
from lz78 import encoded_sequence_from_bytes, spa_from_file

In [3]:
tf.config.set_visible_devices([], 'GPU')

In [4]:
class PG19DataLoader:
    def __init__(self, data_type: str, start_index: int = 0, batch_size: int = 1, normalize: str = 'none'):
        self.data = tfds.load('pg19', split=data_type, shuffle_files=False)
        self.dataset = (self.data
                        .skip(start_index)
                        .batch(batch_size)
                        .prefetch(tf.data.experimental.AUTOTUNE))
        print(data_type, ": ", len(self.dataset))

    def __len__(self):
        return len(self.dataset)

    def __iter__(self):
        for batch in self.dataset:
            text_bytes = np.frombuffer(batch['book_text'].numpy()[0], dtype=np.uint8)
            text_bytes = text_bytes.tolist()
            yield text_bytes

In [5]:
train_dataloader = PG19DataLoader("train")
val_dataloader = PG19DataLoader("validation")

train :  28602
validation :  50


In [6]:
stdout.flush()
model = spa_from_file("../spa_outputs/pg19_spa.bin")

thread '<unnamed>' panicked at /data/home/nsagan/LZ78-implementation/lz78_rust/crates/lz78/src/spa/lz_transform.rs:266:17:
hi 100_000_000
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: hi 100_000_000

In [7]:
model.get_inference_params()

{'gamma': 0.00390625,
 'adaptive_gamma': 'Disabled',
 'n_ensemble': 10,
 'ensemble_type': 'Average',
 'backshift_parsing': True,
 'backshift_ctx_len': 6,
 'backshift_min_count': 1,
 'lb': 0.0001,
 'temp': 1.0,
 'lb_or_temp_first': 'lb_first'}

In [8]:
model.set_inference_params(ensemble_type='depth', ensemble_n=6, backshift_ctx_len=0, lb=1e-5)

In [9]:
model.reset_state()

In [22]:
loss = 0
n_bytes = 0
for (i, inf_batch) in tqdm(enumerate(val_dataloader), desc="Validation"):
    stdout.flush()
    new_loss = model.compute_test_loss(Sequence(inf_batch, alphabet_size=256))
    print(new_loss / len(inf_batch))
    loss += new_loss
    n_bytes += len(inf_batch)
    model.reset_state()

    if i > 5:
        break

print(f"total: {loss / n_bytes}, ppl: {2**(loss / n_bytes)}")

Validation: 0it [00:00, ?it/s]

In [7]:
model.get_inference_params()

{'gamma': 0.5,
 'adaptive_gamma': 'Disabled',
 'n_ensemble': 1,
 'ensemble_type': 'Disabled',
 'backshift_parsing': True,
 'backshift_ctx_len': 5,
 'backshift_min_count': 1,
 'lb': 0.0001,
 'temp': 1.0,
 'lb_or_temp_first': 'lb_first'}

In [8]:
model.set_inference_params(ensemble_type="average", ensemble_n=10, backshift_ctx_len=2, backshift_min_count=0, lb=1e-5, temp=1, gamma=1/256)

In [10]:
for inf_batch in tqdm(val_dataloader, desc="Validation"):
    stdout.flush()
    model.compute_test_loss(Sequence(inf_batch, alphabet_size=256))
    model.reset_state()
    # val_byte += len(inf_batch)

Validation:  12%|█▏        | 6/50 [00:52<06:16,  8.55s/it]

In [10]:
stdout.flush()
total_byte = 0
for trn_iter, batch in enumerate(tqdm(train_dataloader, desc="Building LZ tree"), start=1):
    # build LZ model only 1 epoch
    stdout.flush()
    model.train_on_block(Sequence(batch, alphabet_size=256))
    model.reset_state()
    total_byte += len(batch)

    if trn_iter % 10 == 1:
        print("Total bytes: ", total_byte, "; Training log loss: ", model.get_normalized_log_loss())
        print("Running inference")
        val_loss = 0
        val_byte = 0

        for inf_batch in tqdm(val_dataloader, desc="Validation"):
            stdout.flush()
            val_loss += model.compute_test_loss(Sequence(inf_batch, alphabet_size=256))
            model.reset_state()
            val_byte += len(inf_batch)
        print("Inference log loss: ", val_loss / val_byte)

Building LZ tree:   0%|          | 0/28602 [00:00<?, ?it/s]

Total bytes:  125104 ; Training log loss:  4.554748789335367
Running inference


Building LZ tree:   0%|          | 0/28602 [00:17<?, ?it/s]


KeyboardInterrupt: 